**Module 1:**

In [ ]:

import numpy as np
import pandas as pd

def read_csv():
    # Method to read the CSV file "Hospital_patients_datasets.csv" using pandas.
    # Returns: Pandas DataFrame containing the data from the CSV file.
    ds = pd.read_csv("Hospital_patients_datasets.csv")
    return ds

def check_duplicates():
    ds = read_csv()
    # Method to check for duplicate rows in the DataFrame.
    # Returns: The number of duplicated rows found in the DataFrame.
    return ds.duplicated().sum()

def check_null_values():
    ds = read_csv()
    # Method to check for null (missing) values in the DataFrame.
    # Returns: A pandas Series indicating the count of null values for each column in the DataFrame.
    return ds.isnull().sum()


def converting_dtype():
    ds = read_csv()  # Make sure to define read_csv() function

    # Convert 'ScheduledDay' column to datetime with date-only information
    ds['ScheduledDay'] = pd.to_datetime(ds['ScheduledDay'], utc=True).dt.tz_convert(None).dt.normalize()

    # Convert 'AppointmentDay' column to datetime with date-only information
    ds['AppointmentDay'] = pd.to_datetime(ds['AppointmentDay'], utc=True).dt.tz_convert(None).dt.normalize()

    return ds

def rename_columns():
    ds = converting_dtype()
    # Method to rename some columns in the DataFrame.
    # Returns: DataFrame with certain column names changed to new names.
    ds.rename(columns={
        'Hipertension': 'Hypertension',
        'Handcap': 'Handicap',
        'SMS_received': 'SMSRecevied',  # Corrected column name here
        'No-show': 'NoShow'
    }, inplace=True)
    return ds


# Call the functions in sequence to perform the desired operations

# Task 1: Read the CSV and get the dataset
dataset = read_csv()

# Task 2: Check for duplicate values
duplicates_count = check_duplicates()

# Task 3: Check for null values
null_values_count = check_null_values()

# Task 4: Convert data types
dataset_converted = converting_dtype()

# Task 5: Rename columns
dataset_renamed = rename_columns()




**Module 2:**

In [ ]:
import numpy as np
import pandas as pd
import module1 as m1

def drop_columns():
    ds = m1.rename_columns()
    # Method to drop unnecessary columns from the DataFrame.
    # Returns: DataFrame with specified columns dropped.
    ds.drop(columns=['PatientId', 'AppointmentID', 'Neighbourhood'], inplace=True)
    return ds

def create_bin():
    ds = drop_columns()
    # First drop rows with Age == 0
    ds = ds[ds['Age'] != 0]

    # Generating labels for age intervals (e.g., '1 - 20', '21 - 40', etc.)
    labels = ["{0} - {1}".format(i, i + 20) for i in range(1, 118, 20)]

    # Using the pd.cut() function to categorize ages into groups
    ds['Age_group'] = pd.cut(ds['Age'], bins=range(1, 130, 20), right=False, labels=labels)
    return ds

def drop():
    ds = create_bin()
    # Method to drop the original 'Age' column from the DataFrame.
    # Returns: DataFrame with the 'Age' column dropped.
    ds.drop(columns=['Age'], inplace=True)
    return ds

def convert():
    ds = drop()
    # Method to convert 'NoShow' values into binary values (1 for 'Yes' and 0 for 'No').
    # Returns: DataFrame with 'NoShow' column values converted to 1s and 0s.
    ds['NoShow'] = ds['NoShow'].apply(lambda x: 1 if x == 'Yes' else 0)
    return ds

def export_the_dataset():
    df = convert()
    # Write the cleaned dataset to a new CSV file named 'patients.csv'
    df.to_csv('patients.csv', index=False)
    return df

# TASK 6: Load the Cleaned dataset 'patients.csv' to the database provided.
# Please follow the instructions in Task 5 description and complete the task as per it.
# To run this task, click on the terminal and click on the run project



**Module 3 SQL:**



In [ ]:
SELECT COUNT(*) AS total_values
FROM patients;


SELECT AppointmentDay, COUNT(*) AS appointment_count
FROM patients
GROUP BY AppointmentDay;


SELECT ROUND(AVG(appointment_count)) AS avg_appointments_per_day
FROM (
    SELECT AppointmentDay, COUNT(*) AS appointment_count
    FROM patients
    GROUP BY AppointmentDay
) AS counts;


SELECT AppointmentDay, COUNT(*) AS appointment_count
FROM patients
GROUP BY AppointmentDay
ORDER BY appointment_count DESC
LIMIT 1;


SELECT Month, ROUND(AVG(appointment_count)) AS Appointments_Count
FROM (
    SELECT DATE_FORMAT(AppointmentDay, '%Y-%m') AS Month, COUNT(*) AS appointment_count
    FROM patients
    GROUP BY Month
) AS counts
GROUP BY Month;


SELECT DATE_FORMAT(AppointmentDay, '%Y-%m') AS month,
       SUM(appointment_count) AS total_appointments
FROM (
    SELECT AppointmentDay, COUNT(*) AS appointment_count
    FROM patients
    GROUP BY AppointmentDay
) AS counts
GROUP BY month
ORDER BY total_appointments DESC
LIMIT 1;


SELECT
    YEAR(AppointmentDay) AS Year,
    WEEK(AppointmentDay) AS Week,
    ROUND(AVG(appointment_count)) AS Appointments_Count
FROM (
    SELECT AppointmentDay, COUNT(*) AS appointment_count
    FROM patients
    GROUP BY YEAR(AppointmentDay), WEEK(AppointmentDay)
) AS counts
GROUP BY Year, Week;


SELECT
    YEAR(AppointmentDay) AS Year,
    WEEK(AppointmentDay) AS Week,
    MAX(appointment_count) AS Appointments_Count
FROM (
    SELECT AppointmentDay, COUNT(*) AS appointment_count
    FROM patients
    GROUP BY YEAR(AppointmentDay), WEEK(AppointmentDay)
) AS counts
GROUP BY Year, Week
ORDER BY Appointments_Count DESC
LIMIT 1;


SELECT Gender, COUNT(*) AS appointment_count
FROM patients
GROUP BY Gender;


SELECT DAYNAME(AppointmentDay) AS weekday, COUNT(*) AS appointment_count
FROM patients
GROUP BY weekday
ORDER BY appointment_count DESC;


SELECT ROUND(AVG(DATEDIFF(AppointmentDay, ScheduledDay))) AS avg_days_between
FROM patients;
